In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import seaborn as sns

In [ ]:
az.style.use('arviz-white')
%config Completer.use_jedi = False
np.random.seed(1846)

In [ ]:
X = np.random.uniform(low=0, high=1.0, size=(100, 10))
f_x = 10*np.sin(np.pi*X[:,0]*X[:,1]) + 20*(X[:,2]-0.5)**2 + 10*X[:,3] + 5*X[:,4]
Y = np.random.normal(f_x, 1)

In [ ]:
alphas = [0.1, 0.25, 0.5]
idatas_at = {'10':{'0.1':{}, '0.25':{}, '0.5':{}},
             '20':{'0.1':{}, '0.25':{}, '0.5':{}},
             '50':{'0.1':{}, '0.25':{}, '0.5':{}},
             '100':{'0.1':{}, '0.25':{}, '0.5':{}},
             '200':{'0.1':{}, '0.25':{}, '0.5':{}}}
trees = [10, 20, 50, 100, 200]
for m in trees:
    for alpha in alphas:
        with pm.Model(rng_seeder=678) as model:
            μ = pm.BART('μ', X, Y, m=m, alpha=alpha) 
            σ = pm.HalfNormal('σ', 1)
            y = pm.Normal('y', μ, σ, observed=Y)
            idata = pm.sample(chains=4, random_seed=678)
            idatas_at[str(m)][str(alpha)] = idata

In [ ]:
_, axes = plt.subplots(1, 5, figsize=(10, 3),sharey=True)

for i, ax in enumerate(axes.ravel()):
    m = trees[i]
    tmp_df = pd.DataFrame()
    for alpha in alphas:
        tmp = idatas_at[str(m)][str(alpha)]['posterior']['μ'].stack(samples=["chain",  "draw"])
        mu_mean = np.mean(tmp, 1)
        tmp_df[str(alpha)] = np.array(mu_mean)
    if i == 0:
        ax = sns.boxplot(data=tmp_df, ax=axes[i])
        ax.set_title(f'm = {m}')
        ax.set_ylabel(fr'$\mu$', rotation='horizontal')
        ax.set_xlabel(fr'$\alpha$')
    else:
        ax = sns.boxplot(data=tmp_df, ax=axes[i])
        ax.set_title(f'm = {m}')
        ax.set_xlabel(fr'$\alpha$')

plt.savefig('boxplots-alphas.png')